In [17]:
!pip install pycaret

In [18]:
!pip install botorch

In [19]:
import sklearn
import scipy
from pycaret.regression import *
import botorch
import torch
import gpytorch
import pandas as pd
import numpy as np

# Linear Regression com normalização

In [20]:
df=pd.DataFrame([["AISI_1020",0.5,99999,99999,5,9999999,213,53,0.0055,0.34,0.81,2.70E-03,206000,0.28],
              ["AISI_1045",0.5,99999,99999,5,9999999,553,600,0.0134,0.234,1.000,2.70E-03,210000,0.269],
              ["AISI_4340",0.5,99999,99999,5,9999999,792,510,0.014,1.030,0.26,2.70E-03,209000,0.28],
              ["AISI_4340_52_HRC",0.5,99999,99999,5,9999999,1638,1034,0.0058,0.46,1.003,2.70E-03,222000,0.29],
              ["STEEL_304",0.5,99999,99999,5,9999999,310,1000,0.07,0.65,1.000,2.70E-03,209000,0.28],
              ["STEEL_52100",0.5,99999,99999,5,9999999,774.78,134,0.0018,0.37,3.171,2.70E-03,222000,0.29]],
              columns=["material","depth","vc","rake_angle","clearance_angle","expected_cut_force",
                        "A","B","C","n","m","densidade","young_modulus","poisson_coef"])
df

,material,depth,vc,rake_angle,clearance_angle,expected_cut_force,A,B,C,n,m,densidade,young_modulus,poisson_coef
0,AISI_1020,0.5,99999,99999,5,9999999,213.00,53,0.0055,0.340,0.810,0.0027,206000,0.280
1,AISI_1045,0.5,99999,99999,5,9999999,553.00,600,0.0134,0.234,1.000,0.0027,210000,0.269
2,AISI_4340,0.5,99999,99999,5,9999999,792.00,510,0.0140,1.030,0.260,0.0027,209000,0.280
3,AISI_4340_52_HRC,0.5,99999,99999,5,9999999,1638.00,1034,0.0058,0.460,1.003,0.0027,222000,0.290
4,STEEL_304,0.5,99999,99999,5,9999999,310.00,1000,0.0700,0.650,1.000,0.0027,209000,0.280
5,STEEL_52100,0.5,99999,99999,5,9999999,774.78,134,0.0018,0.370,3.171,0.0027,222000,0.290


In [21]:
def build_input(controlable_input, fixed_input,construct_dict):
    final_input_df=pd.DataFrame()
    for input in controlable_input:
        for counter,parameter in enumerate(construct_dict.keys()):
            fixed_input[parameter]=input[counter]
        final_input_df=pd.concat([final_input_df,fixed_input])
    return final_input_df

In [22]:
construct_dict={"rake_angle":{"sequence_len":4,"max_bound":10,"lower_bound":-10,"type":"int"},
                "vc":{"sequence_len":10,"max_bound":500,"lower_bound":30,"type":"int"},
                #"depth":{"sequence_len":4,"max_bound":2,"lower_bound":0.1,"type":"float"}
                }

In [23]:
degree_to_radian=lambda x: x*np.pi/180
radians_to_degree=lambda x: x*180/np.pi

# Minimização da força de corte

In [24]:
lasso_v0=load_model("models/lasso_v0")
lgbm_v0=load_model("models/lgbm_v0")
lgbm_v1=load_model("models/lgbm_v1")

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [25]:
def objective_func(x,iloc_=0,model="lasso"):
    #x -> [[rake_angle,vc]]
    x=[x]
    inputs=build_input(x,df.iloc[[iloc_]],construct_dict)
    if model=="lasso":
      model_art=lasso_v0
    else:
      model_art=lgbm_v1
    cut_force=(predict_model(model_art,inputs,verbose=False).drop_duplicates()["prediction_label"].values)
    return cut_force[0]

### Força bruta

In [26]:
import itertools
from itertools import permutations

# initialize lists
list_1 = [str(x) for x in np.linspace(-15,15,10)]
list_2 = [str(x) for x in np.linspace(3000,5000,3)]

unique_combinations = []

permut = itertools.permutations(list_1, len(list_2))

for comb in permut:
    zipped = zip(comb, list_2)
    unique_combinations.append(list(zipped))

flat_list = [item for sublist in unique_combinations for item in sublist]

In [27]:
len(flat_list)

2160

In [41]:
from tqdm import tqdm
dict_={"rake_angle":[],"vc":[],"fcut_lasso":[],"fcut_lgbm":[],"material":[]}
count=0
for i in tqdm(flat_list):
    count+=1
    for j in range(df.shape[0]):
        dict_["rake_angle"].append(i[0])
        dict_["vc"].append(i[1])
        dict_["fcut_lasso"].append(objective_func(i,j,"lasso"))
        dict_["fcut_lgbm"].append(objective_func(i,j,"lgbm"))
        dict_["material"].append(df.iloc[j]["material"])
    #if count>5:
      #break
results=pd.DataFrame.from_dict(dict_)
results.head()

  0%|          | 5/2160 [00:26<3:13:30,  5.39s/it]


,rake_angle,vc,fcut_lasso,fcut_lgbm,material
0,-15.0,3000.0,1394.011743,1040.572327,AISI_1020
1,-15.0,3000.0,5788.703390,5791.133299,AISI_1045
2,-15.0,3000.0,6764.847051,6265.977035,AISI_4340
3,-15.0,3000.0,22627.160543,6272.179365,AISI_4340_52_HRC
4,-15.0,3000.0,2629.079749,2135.276159,STEEL_304


In [42]:
results.sort_values("fcut_lasso").drop_duplicates("material")

,rake_angle,vc,fcut_lasso,fcut_lgbm,material
30,-5.0,5000.0,782.958232,982.513731,AISI_1020
34,-5.0,5000.0,1994.438806,1992.931096,STEEL_304
35,-5.0,5000.0,2776.261132,2876.947304,STEEL_52100
31,-5.0,5000.0,5012.107161,4724.354471,AISI_1045
32,-5.0,5000.0,5994.457605,5896.526062,AISI_4340
33,-5.0,5000.0,21526.674905,5877.913717,AISI_4340_52_HRC


In [43]:
results.sort_values("fcut_lgbm").drop_duplicates("material")

,rake_angle,vc,fcut_lasso,fcut_lgbm,material
24,-11.666666666666666,4000.0,1126.299326,940.504821,AISI_1020
16,-8.333333333333332,5000.0,2104.613743,1979.275071,STEEL_304
17,-8.333333333333332,5000.0,2925.707357,2843.290463,STEEL_52100
31,-5.0,5000.0,5012.107161,4724.354471,AISI_1045
14,-8.333333333333332,5000.0,6149.882043,5855.942188,AISI_4340
15,-8.333333333333332,5000.0,21792.131407,5876.093904,AISI_4340_52_HRC


In [45]:
import plotly.express as px

results["vc_m/min"]=results["vc"].astype(float)*60/1000
results["rake_angle"]=results["rake_angle"].astype(float)
results["fcut_lasso"]=results["fcut_lasso"].astype(float)
results["fcut_lgbm"]=results["fcut_lgbm"].astype(float)
for material in results["material"].unique():
  for model_ in ["lasso","lgbm"]:
    fig = px.scatter(results[results["material"]==material],
                    x="rake_angle",
                    y=f"fcut_{model_}",
                    color="vc_m/min",
                    title=f"Força de corte prevista por velocidade de corte e ângulo de saída para o {material} com {model_}",
                    )

    fig.update_layout(
                    xaxis_title="Ângulo de saída [°]",
                    yaxis_title="Força de corte [N]",
                    legend_title="Velocidade de corte [m/min]",
      )
    fig.show()

## Método de minimização

In [ ]:
from scipy.optimize import minimize

bounds=[(-15,15),(3000,5000)]

results=pd.DataFrame()
count=0
for model_ in ["lgbm","lasso"]:
  for idx_mat,material in enumerate(df["material"].unique()):
    count+=1
    output=minimize(objective_func,args=(idx_mat,model_),x0=[0,3500],method="Powell",bounds=bounds)
    results=pd.concat([
        results,
        pd.DataFrame(output.items()).set_index(0).T.assign(material=df["material"].unique()[idx_mat],
                                                           model=model_)
        ])

Exception ignored in: <function _xla_gc_callback at 0x7f3e1d266440>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
Exception ignored in: <function _xla_gc_callback at 0x7f3e1d266440>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
Exception ignored in: <function _xla_gc_callback at 0x7f3e1d266440>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
Exception ignored in: <function _xla_gc_callback at 0x7f3e1d266440>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_

In [ ]:
results